In [ ]:
#SQQNN Binary Classification for MNIST
# Import required libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from QBCNetwork import *  # Custom Quantum Bayesian Network implementation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics
from scipy.fftpack import fft2, dct  # For Discrete Cosine Transform (DCT)
from KFolderCV import *  # Custom K-Fold Cross Validation implementation

def get_matrix_labels(idx, nk):
    """
    Convert class indices to one-hot encoded matrix
    Args:
        idx: Array of class indices
        nk: Number of classes
    Returns:
        One-hot encoded matrix
    """
    n = np.size(idx, 0)
    mtx = np.zeros((n, nk))
    for i in range(0, n):
        mtx[i, idx[i]] = 1
    return mtx    

# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Combine training and test data for full dataset analysis
x_data = np.concatenate((x_train, x_test), axis=0)
y_data = np.concatenate((y_train, y_test), axis=0)

total_size = x_data.shape[0]

# Normalize pixel values to [0,1] range and convert to float
x_data = (x_data.astype(float)/255 )

# Apply 2D Discrete Cosine Transform (DCT) to each image and flatten to 1D array
# This transforms spatial domain images to frequency domain features
x_data = np.array([dct(dct(x_data[i,:,:], axis=0, norm='ortho'), axis=1, norm='ortho').flatten() 
                  for i in range(total_size)])       

# Initialize parameters for pairwise classification
nk = 1  # Number of output units (binary classification)

# Perform pairwise classification between all digit combinations
for i in range(0, 10):  # For each digit 0-9
    for j in range(i+1, 10):  # Compare with all higher digits
        if(i != j):
            # Define the two classes to compare
            class1 = i
            class2 = j
            
            # Get indices of samples belonging to either class
            idx = np.where((y_data == class1) | (y_data == class2))[0]
            
            # Get corresponding features and labels
            nx = x_data[idx,:]
            
            # Normalize labels to 0 (class1) and 1 (class2)
            ny = (y_data[idx]-i)/(j-i)
            
            # Perform 10-fold cross validation
            kf = KFolderCV(10)
            
            # Get evaluation metrics from cross-validation
            # ma: mean accuracy, sa: std accuracy
            # mp: mean precision, sp: std precision
            # mse: mean sensitivity (recall), sse: std sensitivity
            # msp: mean specificity, ssp: std specificity
            # mf: mean F1-score, sf: std F1-score
            ma, sa, mp, sp, mse, sse, msp, ssp, mf, sf = kf.cross_validate(nx, ny, nk)
            
            # Print evaluation results for this pair of classes
            print("Classes: ", class1, " vs ", class2)
            print(f"Accuracy: {ma:.3f} (±{sa:.3f})")
            print(f"Precision: {mp:.3f} (±{sp:.3f})")
            print(f"Sensitivity (Recall): {mse:.3f} (±{sse:.3f})")
            print(f"Specificity: {msp:.3f} (±{ssp:.3f})")
            print(f"F1 Score: {mf:.3f} (±{sf:.3f})")
            print("----------------------------------------")

2025-03-27 17:08:04.026354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743106084.037072   34716 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743106084.040293   34716 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743106084.049380   34716 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743106084.049391   34716 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743106084.049392   34716 computation_placer.cc:177] computation placer alr

Classes:  0  vs  1
Accuracy: 0.999 (±0.001)
Precision: 0.999 (±0.001)
Sensitivity (Recall): 0.999 (±0.001)
Specificity: 0.999 (±0.001)
F1 Score: 0.999 (±0.001)
----------------------------------------
Classes:  0  vs  2
Accuracy: 0.995 (±0.002)
Precision: 0.997 (±0.003)
Sensitivity (Recall): 0.993 (±0.002)
Specificity: 0.993 (±0.002)
F1 Score: 0.995 (±0.002)
----------------------------------------
